In [ ]:
import os
import sys
sys.path.insert(0, "../..")
sys.path.insert(0, "../../../acquire")
import ipywidgets as widgets

from HUGS.Processing import search

from Acquire.Client import User, Drive, Service, PAR, Authorisation, StorageCreds

In [ ]:
# Autoreload modules before executing code, useful during development
%load_ext autoreload
%autoreload 2

## Start with uploading and processing of the data

In [ ]:
base_url = "https://hugs.acquire-aaai.com/t"
hugs_url = "https://hugs.acquire-aaai.com"
user = User(username="gareth", identity_url="%s/identity" % base_url)
response = user.request_login()

Login by visiting: https://login.acquire-aaai.com?id=a0-a6/46.8f.b5.ff
(please check that this page displays the message 'big sheep climb quickly')


Check we're logged in

In [ ]:
user.wait_for_login()

True

In [ ]:
filename = "bsd.picarro.1minute.248m.dat"
dir_path = os.path.abspath("")
test_data = "../../test/data/proc_test_data/CRDS"
filepath = os.path.join(dir_path, test_data, filename)

hugs = Service(service_url="%s/hugs" % base_url)

creds = StorageCreds(user=user, service_url="%s/storage" % base_url)
drive = Drive(creds=creds, name="test_drive")
filemeta = drive.upload(filepath)
par = PAR(location=filemeta.location(), user=user)

par_secret = hugs.encrypt_data(par.secret())
auth = Authorisation(resource="process", user=user)

In [ ]:
def upload_file(authenticated_user, filepath):
    """ Upload a file to the object store
    
        Args:
            filepath (str): Path to file to upload
        Returns:
            None    
    """
    creds = StorageCreds(user=authenticated_user, service_url="%s/storage" % base_url)
    drive = Drive(creds=creds, name="test_drive")
    filemeta = drive.upload(filepath)
    par = PAR(location=filemeta.location(), user=user)

    par_secret = hugs.encrypt_data(par.secret())
    auth = Authorisation(resource="process", user=user)

In [ ]:
def get_CRDS_path(filename):
    dir_path = os.path.abspath("")
    test_data = "../../test/data/proc_test_data/CRDS"
    return os.path.join(dir_path, test_data, filename)

### Upload files for processing


Upload Bilsdale data

In [ ]:
bsd_path = get_CRDS_path("bsd.picarro.1minute.248m.dat")
upload_file(authenticated_user=user, filepath=bsd_path)


Clear the Datasources from CRDS and GC objects (currently Datasource UUIDs are just being randomly generated and we get multiple Datasources holding the same data)

In [ ]:
response = hugs.call_function(function="clear_datasources", args={})
response

In [ ]:
from HUGS.Client import Process

filename = "bsd.picarro.1minute.248m.dat"
filepath = get_CRDS_path(filename)

processing = Process(service_url=base_url)
result = processing.process_files(user=user, files=filepath, data_type="CRDS")
print(result)

{'bsd.picarro.1minute.248m.dat': ['9f01c316-30af-45e1-8c01-0b144ae0ccc6', 'ac0482af-cc0b-4a6a-9cf3-6777d5b66102', '4cb9e3ce-89d4-407f-85a9-63f45471e279']}


Upload some Heathfield data

In [ ]:
filename = "hfd.picarro.1minute.100m_min.dat"
filepath = get_CRDS_path(filename)
result = processing.process_files(user=user, files=filepath, data_type="CRDS")
print(result)

{'hfd.picarro.1minute.100m_min.dat': ['fbc6f867-5665-43a2-909d-2d5827252474', '1c392920-2d88-43a7-8bae-4ed6a2e46886', '2ed2fa7e-7192-4e52-996d-ecc4c4a0e009']}


In [ ]:
from HUGS.Processing import search

search_results = None

search_terms = widgets.Text(
                    value="",
                    placeholder = "Search",
                    description = "Search terms:",
                    disabled = False
)

locations = widgets.Text(
                value="",
                placeholder = "BSD, HFD",
                description = "Locations:",
                disabled = False
)

data_type = widgets.Dropdown(
                options=["CRDS", "GC"], 
                value="CRDS",
                description="Data type",
                disabled=False
    )



layout=widgets.Layout(display = "flex", width = "50%")
search_button = widgets.Button(description="Search", button_style="success", \
                               layout=widgets.Layout(flex='1 1 0%', width='25%'))

start_picker = widgets.DatePicker(
                    description='Start date',
                    disabled=False
)

end_picker = widgets.DatePicker(
                    description='End date',
                    disabled=False
)


box = widgets.VBox(children=[search_terms, locations, start_picker, end_picker, data_type,
                                 search_button], layout=layout)



def call_search(x):
    """ Call the search function and pass it the values 
        in the text boxes
            
    """
    from datetime import datetime
    from Acquire.ObjectStore import datetime_to_string
    from HUGS.Client import Search
    start = datetime(1970, 1,1) # datetime.combine(start_picker.value, datetime.min.time())
    end = datetime.now() # datetime.combine(end_picker.value, datetime.min.time())
    
    split_search_terms = search_terms.value.replace(" ", "").split(",")
    split_locations = locations.value.replace(" ", "").split(",")
    
    global search_results
    search = Search(service_url=base_url)
    search_results = search.search(search_terms=split_search_terms, locations=split_locations, data_type=data_type.value, start_datetime=start, end_datetime=end)

search_button.on_click(call_search)




In [ ]:
box

['co', 'co2'] ['bsd']


In [ ]:
print(search_results)

{'bsd_co': ['data/uuid/4cb9e3ce-89d4-407f-85a9-63f45471e279/2014-01-30T10:52:30_2014-01-30T14:20:30'], 'bsd_co2': ['data/uuid/ac0482af-cc0b-4a6a-9cf3-6777d5b66102/2014-01-30T10:52:30_2014-01-30T14:20:30']}


Now take the result dictionary, parse the dictionary into a blocks that can be selected by the user

In [ ]:
def parse_results(results):
    """ Split the keys into a list of each key and the date that the data covers
        
        Args:
            results (dict): Dictionary of search results
        Returns:
            list (tuple): List of date, data key list pairs
    """
    date_keys = {}
    for key in results.keys():
        keys = sorted(results[key])
        start_key = keys[0]
        end_key = keys[-1]
        # Get the first and last dates from the keys in the search results
        start_date = start_key.split("/")[-1].split("_")[0]
        end_date = end_key.split("/")[-1].split("_")[-1]
        
        dates_covered = start_date + "_" + end_date
        
        date_keys[key] = {"dates": dates_covered, "keys": keys}
        
    return date_keys

Site, gas, dates and checkbox

In [ ]:
date_keys = parse_results(search_results)
print(date_keys)

{'bsd_co': {'dates': '2014-01-30T10:52:30_2014-01-30T14:20:30', 'keys': ['data/uuid/4cb9e3ce-89d4-407f-85a9-63f45471e279/2014-01-30T10:52:30_2014-01-30T14:20:30']}, 'bsd_co2': {'dates': '2014-01-30T10:52:30_2014-01-30T14:20:30', 'keys': ['data/uuid/ac0482af-cc0b-4a6a-9cf3-6777d5b66102/2014-01-30T10:52:30_2014-01-30T14:20:30']}}


In [ ]:
from ipywidgets import Button, ButtonStyle, Checkbox, HTML, Label, HBox, VBox

table_style = {'description_width': 'initial'}
table_layout = {'width':'100px', 'min_width':'100px', 'height':'28px', 'min_height':'28px'}
date_layout = {'width':'275px', 'min_width':'200px', 'height':'28px', 'min_height':'28px'}
checkbox_layout = {'width':'100px', 'min_width':'100px', 'height':'28px', 'min_height':'28px'}
statusbar_layout = {'width':'250px', 'min_width':'250px', 'height':'28px', 'min_height':'28px'}

# row_layout = {'width':'200px', 'min_width':'200px'}

header_label_site = HTML(value=f"<b>Site</b>", layout=table_layout)
header_label_gas = HTML(value=f"<b>Gas</b>", layout=table_layout)
header_label_dates = HTML(value=f"<b>Dates</b>", layout=date_layout)
header_label_select = HTML(value=f"<b>Select</b>", layout=checkbox_layout)

checkbox_objects = []
search_keys = []

site_labels = []
date_labels = []
gas_labels = []
for key in date_keys:
    # Create the checkboxes
    checkbox = Checkbox(value=False)
    checkbox_objects.append(checkbox)
    search_keys.append(key)
    
    dates = date_keys[key]["dates"].replace("_", " to ").replace("T", " ")
    date_label = Label(value=dates, layout=date_layout)
    
    split_key = key.split("_")
    site_name = split_key[0].upper()
    gas_name = split_key[1].upper()
    
    gas_label = Label(value=gas_name, layout=table_layout)
    site_label = Label(value=site_name, layout=table_layout)

    date_labels.append(date_label)
    site_labels.append(site_label)
    gas_labels.append(gas_label)

    
arg_dict = {search_keys[i]: checkbox for i, checkbox in enumerate(checkbox_objects)}

header_box = HBox(children=[header_label_site, header_label_gas, header_label_dates, header_label_select])

site_vbox = VBox(children=site_labels)
gas_vbox = VBox(children=gas_labels)
dates_vbox = VBox(children=date_labels)
checkbox_vbox = VBox(children=checkbox_objects)

dynamic_box = HBox(children=[site_vbox, gas_vbox, dates_vbox, checkbox_vbox])

download_button = Button(description="Download", button_style="success", layout=table_layout)
download_box = HBox(children=[download_button])

status_bar = HTML(value="Status: Waiting...", layout=statusbar_layout)

complete = VBox(children=[header_box, dynamic_box, download_box, status_bar])

selected_data = []
def select_data(**kwargs):
    selected_data.clear()
    
    for key in kwargs:
        if kwargs[key] is True:
            selected_data.append(key)

def update_statusbar(text):
    status_bar.value = F"Status: {text}"
    
data = None
def download_data(args):
    """ Download the data in the selected keys from the object store
    
        Args:
            ?
        Returns:
            Pandas.Dataframe of selected data
    """
    from HUGS.Client import Retrieve
    from pandas import read_json as pd_read_json
    
    update_statusbar("Downloading...")
    
    download_keys = {key: date_keys[key]["keys"] for key in selected_data}
    
    retrieve = Retrieve(service_url=base_url)
    
    global data
    data = retrieve.retrieve(keys=download_keys)
    
    # Conver the JSON into Dataframes
    for key in data:
        data[key] = pd_read_json(data[key])
    
    # Update the status bar
    if len(data) > 0:
        update_statusbar("Download complete")
    else:
        update_statusbar("No data downloaded")
            
download_button.on_click(download_data)
out = widgets.interactive_output(select_data, arg_dict)

display(complete, out)

Output()

In [ ]:
data.keys()

dict_keys(['bsd_co'])

Retrieve the data from the object store

In [ ]:
selected_data

['bsd_co']

In [ ]:
data

{'bsd_co':                      co count  co stdev  co n_meas
 2014-01-30 10:52:30    204.62     6.232         26
 2014-01-30 10:53:30    200.78     5.934         26
 2014-01-30 10:54:30    201.54     5.176         25
 2014-01-30 10:55:30    203.39     6.535         26
 2014-01-30 10:56:30    201.87     6.345         25
 ...                       ...       ...        ...
 2014-01-30 14:16:30    197.83     6.410         26
 2014-01-30 14:17:30    197.37     6.194         25
 2014-01-30 14:18:30    197.52     8.251         26
 2014-01-30 14:19:30    196.92     6.031         26
 2014-01-30 14:20:30    196.35     6.879         25
 
 [76 rows x 3 columns]}

Plot the data we've selected

Select the data from the downloaded data using the checkboxes and then click plot to update the figure

In [ ]:
# Create some checkboxes
plot_checkboxes = []
plot_keys = [] 

for key in data:
    # Create a more readable description
    desc = " ".join(key.split("_")).upper()
    plot_keys.append(key)
    plot_checkboxes.append(Checkbox(description=desc, value=False))
    
select_instruction = HTML(value="Select data: ", layout=table_layout)
plot_button = Button(description="Plot", button_style="success", layout=table_layout)

select_box = HBox(children=[select_instruction])
checkbox_box = HBox(children=plot_checkboxes)
horiz_select = HBox(children=[select_box, checkbox_box])
plot_box = HBox(children=[plot_button])
ui_box = VBox(children=[horiz_select, plot_box])

arg_dict = {plot_keys[i]: checkbox for i, checkbox in enumerate(plot_checkboxes)}

selected_data = []
def select_data(**kwargs):
    selected_data.clear()
    
    for key in kwargs:
        if kwargs[key] is True:
            selected_data.append(key)
    
    print(selected_data)
    
# Link the checkbox selection with the selected dataframes to plot
out = widgets.interactive_output(select_data, arg_dict)

output = widgets.Output()

def plot_data(an_arg):
    """ Each key in the data dict is a dataframe
    
    """
    from bqplot import pyplot as plt
    from bqplot import (DateScale, LinearScale, Axis, Lines, Figure)
    # Here take the keys in the selected data list and use them to
    # access the Dataframes to plot
    # Use the same axes. Can have a button to create new plots etc in the future
    
    # TODO - change this to take the data directly from the dict?
    plot_data = [data[x] for x in selected_data]
        
    # Setup the axes
    x_scale = DateScale()
    y_scale = LinearScale()
    
    ax = Axis(label="Date", scale=x_scale)
    # TODO - this could be updated depending on what's being plot
    ay = Axis(label="Count", scale=y_scale, orientation="vertical")
    
    scales = {"x": x_scale, "y": y_scale}
    lines = [Lines(x=d.index.values, y=d["co count"], scales=scales) for d in plot_data]
    
    fig = Figure(marks=lines, axes=[ax, ay], title="A sweet figure")
    
    display(fig)
        
    
plot_button.on_click(plot_data)

display(ui_box, out)

Output()

Figure(axes=[Axis(label='Date', scale=DateScale()), Axis(label='Count', orientation='vertical', scale=LinearSc…

Figure(axes=[Axis(label='Date', scale=DateScale()), Axis(label='Count', orientation='vertical', scale=LinearSc…

Figure(axes=[Axis(label='Date', scale=DateScale()), Axis(label='Count', orientation='vertical', scale=LinearSc…

Figure(axes=[Axis(label='Date', scale=DateScale()), Axis(label='Count', orientation='vertical', scale=LinearSc…

In [ ]:
plot_checkboxes

In [ ]:
from ipywidgets import Checkbox, interactive
from IPython.display import display

l = ["Dog", "Cat", "Mouse"]
chk = [Checkbox(description=a) for a in l]

def updatePlot(**kwargs):
    print [k,v for k, v in kwargs.items()]

interact(updatePlot, **{c.description: c.value for c in chk})

In [ ]:
# imports
%matplotlib inline

from ipywidgets import interactive
import pandas as pd
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt

# Sample data
np.random.seed(123)
rows = 50
dfx = pd.DataFrame(np.random.randint(90,110,size=(rows, 1)), columns=['Variable X'])
dfy = pd.DataFrame(np.random.randint(25,68,size=(rows, 1)), columns=['Variable Y'])
dfz = pd.DataFrame(np.random.randint(60,70,size=(rows, 1)), columns=['Variable Z'])

df = pd.concat([dfx,dfy,dfz], axis = 1)
#jtplot.style()

import ipywidgets as widgets
from IPython.display import display

opts = df.columns.values

selector = widgets.SelectMultiple(
options=opts,
value=[opts[1]],
rows=len(opts),
description='Variables',
disabled=False)

output = widgets.Output()

display(selector)
display(output)

def multiplot(widg):
    choices = widg['new']
    data = df.loc[:, choices] if choices else df
    output.clear_output(wait=True)
    with output:
        ax = data.plot()
        plt.show()

selector.observe(multiplot, names='value')

Now we can plot some of the data

In [ ]:
import pandas as pd
# co2_data = pd.read_json(data["results"]["bsd_co2"])
co_data = pd.read_json(data["results"]["bsd_ch4"])


In [ ]:
co_data.head(10)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

fig = plt.figure()
# ax = fig.add_subplot(211)
# ax.plot(co2_data.index.values, co2_data["co2 count"], color = "#4e79a7" )
ax = fig.add_subplot(111)
ax.plot(co_data.index.values, co_data["ch4 count"], color = "#59a14f")

plt.show()

In [ ]:
# Checkboxes for selecting which data we want to plot
# Select the gas, it goes and plots the 
# Simple line plot

In [ ]:
from ipyleaflet import (
    Map,
    Marker, MarkerCluster, TileLayer, ImageOverlay, GeoJSON,
    Polyline, Polygon, Rectangle, Circle, CircleMarker, Popup,
    SplitMapControl, WidgetControl,
    basemaps, basemap_to_tiles
)

from ipywidgets import HTML

In [ ]:
center = [54.2361, -4.548]
zoom = 5
m = Map(center=center, zoom=zoom)
mark_bsd = Marker(location=(54.942544, -1.369204))
mark_bsd.popup = HTML(value="Bilsdale")

mark_mhd = Marker(location=(53.20,-9.54))
mark_mhd.popup = HTML(value="Macehead")

mark_tac = Marker(location=(52.511, 1.155003))
mark_tac.popup = HTML(value="Tacolneston")

m += mark_bsd
m += mark_mhd
m += mark_tac

display(m)

In [ ]:
m